In [105]:
import requests
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def extract_html(url, webdriver_path="/usr/bin/chromedriver"):
    options = Options()
    options.headless = True  # Set to False if you want to see the browser window
    driver = webdriver.Chrome(executable_path=webdriver_path, options=options)
    driver.get(url)

    try:
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.TAG_NAME, "ul"))
        )
        html = driver.page_source
    except Exception as e:
        print(f"Exception occurred: {e}")
        html = None
    finally:
        driver.quit()

    return html
    
def scrape_jobs_list(soup):
    ul_tag = soup.find('ul')
    if ul_tag:
        p_tags = ul_tag.find_all('p')
        p_texts = [p_tag.get_text() for p_tag in p_tags]
        return p_texts
    else:
        print("No 'ul' tag found.")
        return []

def get_all_profession_links(soup):
    pattern = r'https?://\S+starting-with-[a-z]'
    urls = [link['href'] for link in soup.find_all('a', href=True)]
    proffesion_links = set()

    for url in urls:
        matches = re.findall(pattern, url)
        if matches:
            proffesion_links.add(url)

    return proffesion_links


In [106]:
url = "https://spotterful.com/blog/job-description-template/job-titles-list-a-z"

html = extract_html(url)
soup = BeautifulSoup(html)

/tmp/ipykernel_68251/623631836.py:12: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True  # Set to False if you want to see the browser window
/tmp/ipykernel_68251/623631836.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=webdriver_path, options=options)


<html lang="en">
 <head>
  <script src="https://partner.googleadservices.com/gampad/cookie.js?domain=spotterful.com&amp;callback=_gfp_s_&amp;client=ca-pub-7101372108443523">
  </script>
  <script id="google_shimpl" src="https://pagead2.googlesyndication.com/pagead/managed/js/adsense/m202304120201/show_ads_impl.js?bust=31073823">
  </script>
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-QR0M06N2PM&amp;l=dataLayer&amp;cx=c" type="text/javascript">
  </script>
  <script async="" src="https://www.googletagmanager.com/gtm.js?id=GTM-TLFLPK9">
  </script>
  <script>
   !function(e,t,a,n,g){e[n]=e[n]||[],e[n].push({"gtm.start":(new Date).getTime(),event:"gtm.js"});var m=t.getElementsByTagName(a)[0],r=t.createElement(a);r.async=!0,r.src="https://www.googletagmanager.com/gtm.js?id=GTM-TLFLPK9",m.parentNode.insertBefore(r,m)}(window,document,"script","dataLayer")
  </script>
  <meta charset="utf-8"/>
  <link href="https://fonts.googleapis.com" rel="preconnect"/>
  <link cr

In [107]:
job_list = scrape_jobs_list(soup)
profession_links = get_all_profession_links(soup)
print(len(profession_links))
counter = 0
for link in profession_links:
    # print(f"Iteration: {counter}")
    counter += 1
    html = extract_html(link)
    soup = BeautifulSoup(html, 'html.parser')
    new_jobs = scrape_jobs_list(soup)
    if new_jobs == None:
        print(f"No Jobs found in {link}")
    job_list.extend(new_jobs)

25
Iteration: 0


/tmp/ipykernel_68251/623631836.py:12: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True  # Set to False if you want to see the browser window
/tmp/ipykernel_68251/623631836.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=webdriver_path, options=options)


Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24


In [108]:
jobs = [job for job in job_list if job != ' ' and job !='']
jobs

['ABA Therapist',
 'ABAP Developer',
 'Academic Administrator',
 'Academic Advisor',
 'Academic And Behavior Support Staff',
 'Academic Coach',
 'Academic Coordinator',
 'Academic Counselor',
 'Academic Dean',
 'Academic Director',
 'Academic Tutor',
 'Accident Benefits Representative',
 'Accommodation Service Specialist',
 'Accompanist',
 'Account Administrator',
 'Account Administrator II Fleet',
 'Account Analyst',
 'Account Assistant',
 'Account Associate',
 'Account Based Marketing Manager',
 'Account Coordinator',
 'Account Development Executive',
 'Account Development Manager',
 'Account Development Representative',
 'Account Director',
 'Account Handler',
 'Account Manager',
 'Accounting Administrator',
 'Accounting Analyst',
 'Accounting Assistant',
 'Accounting Associate',
 'Accounting Auditor',
 'Accounting Bookkeeper',
 'Accounting Clerk',
 'Accounting Consultant',
 'Accounting Controller',
 'Accounting Coordinator',
 'Accounting Director',
 'Accounting Generalist',
 'Accou

In [114]:
import pandas as pd

job_list.sort()

df = pd.DataFrame({"Job_Titles": job_list})

df.to_csv("/home/marcuswrrn/Projects/Semantic_Quantification/Semantic_Comparison/Data/Job_Titles.csv", index=False)